# Train and Deploy a Model using Vertex AI

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform --user -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Setting the project ID and region

In [ ]:
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID:", PROJECT_ID)

REGION = "us-central1"

In [ ]:
! gcloud config set project $PROJECT_ID

#### UUID

Some resources like the cloud bucket will need to have a unique name. An easy way to do that is to use a UUID.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

#### Create a Cloud Storage bucket

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Import libraries

In [ ]:
from google.cloud import bigquery

import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

### Initialize Vertex AI SDK and BQ Client for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### TODO: Fetch data from Big Query and create a dataset

In [ ]:
LABEL_COLUMN = "species"

# Define the BigQuery source dataset
BQ_SOURCE = "bigquery-public-data.ml_datasets.penguins"


### TODO: Write the training script

### TODO: Create and submit a training job

### TODO: Deploy the trained model

### TODO: Send a prediction request to the model

### Remember to delete the jobs and models you created to save training costs

In [ ]:
# Delete the training job
job.delete()

# Delete the endpoint and undeploy the model from it
endpoint.delete(force=True)

# Delete the model
model.delete()

## Configure a hyperparameter tuning job

Now that your training application code is containerized, it's time to specify and run the hyperparameter tuning job.

To launch the hyperparameter tuning job, you need to first define the `worker_pool_specs`, which specifies the machine type and Docker image. The following spec defines one `n1-standard-4` machine with one `NVIDIA Tesla T4` GPU as the accelerator.

In [ ]:
# The spec of the worker pools including machine type and Docker image
# Be sure to replace PROJECT_ID in the `image_uri` with your project.

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
        },
    }
]

### Define parameter spec

Next, define the `parameter_spec`, which is a dictionary specifying the parameters you want to optimize. The **dictionary key** is the string you assigned to the command line argument for each hyperparameter, and the **dictionary value** is the parameter specification.

For each hyperparameter, you need to define the `Type` as well as the bounds for the values that the tuning service will try. Hyperparameters can be of type `Double`, `Integer`, `Categorical`, or `Discrete`. If you select the type `Double` or `Integer`, you need to provide a minimum and maximum value. And if you select `Categorical` or `Discrete` you need to provide the values. For the `Double` and `Integer` types, you also need to provide the scaling value. Learn more about [Using an Appropriate Scale](https://www.youtube.com/watch?v=cSoK_6Rkbfg).

In [ ]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_units": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None),
}

The final spec to define is `metric_spec`, which is a dictionary representing the metric to optimize. The dictionary key is the `hyperparameter_metric_tag` that you set in your training application code, and the value is the optimization goal.

In [ ]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

Once the specs are defined, you create a `CustomJob`, which is the common spec that will be used to run your job on each of the hyperparameter tuning trials.

In [ ]:
my_custom_job = aiplatform.CustomJob(
    display_name="horses-humans-sdk-job",
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET_URI,
)

Then, create and run `HyperparameterTuningJob`.

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name="horses-humans-sdk-job",
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=10,
    parallel_trial_count=3,
)

hp_job.run()

There are a few arguments to note:


* **max_trial_count**: You need to put an upper bound on the number of trials the service will run. More trials generally leads to better results, but there will be a point of diminishing returns, after which additional trials have little or no effect on the metric you're trying to optimize. It is a best practice to start with a smaller number of trials and get a sense of how impactful your chosen hyperparameters are before scaling up.

* **parallel_trial_count**: If you use parallel trials, the service provisions multiple training processing clusters. Increasing the number of parallel trials reduces the amount of time the hyperparameter tuning job takes to run; however, it can reduce the effectiveness of the job overall. This is because the default tuning strategy uses results of previous trials to inform the assignment of values in subsequent trials.

* **search_algorithm**: You can set the search algorithm to grid, random, or default (None). If you do not specify an algorithm, as shown in this example, the default option applies Bayesian optimization to search the space of possible hyperparameter values and is the recommended algorithm. Learn more about [Hyperparameter tuning using Bayesian Optimization](https://cloud.google.com/blog/products/ai-machine-learning/hyperparameter-tuning-cloud-machine-learning-engine-using-bayesian-optimization)

## Examine results

Click on the generated link in the output to see your run in the Cloud Console. When the job completes, you will see the results of the tuning trials.

You can sort the results by the optimization metric and then set the hyperparamers in your training application code to the values from the trial with the highest accuracy.

![console_ui_results](tuning_results.png)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete artifact registry repo
! gcloud artifacts repositories delete $REPO_NAME --location $REGION --quiet

delete_custom_job = True
delete_bucket = False

# Delete hptune job
if delete_custom_job:
    try:
        hp_job.delete()
    except Exception as e:
        print(e)

# Delete bucket
if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil rm -r $BUCKET_URI